## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


**1264. Page Recommendations (Medium)**

**Table: Friendship**

| Column Name   | Type    |
|---------------|---------|
| user1_id      | int     |
| user2_id      | int     |

(user1_id, user2_id) is the primary key (combination of columns with unique values) for this table.
Each row of this table indicates that there is a friendship relation between user1_id and user2_id.
 
**Table: Likes**

| Column Name | Type    |
|-------------|---------|
| user_id     | int     |
| page_id     | int     |

(user_id, page_id) is the primary key (combination of columns with unique values) for this table.
Each row of this table indicates that user_id likes page_id.
 
**Write a solution to recommend pages to the user with user_id = 1 using the pages that your friends liked. It should not recommend pages you already liked.**

Return result table in any order without duplicates.

The result format is in the following example.

**Example 1:**

**Input:** 

**Friendship table:**
| user1_id | user2_id |
|----------|----------|
| 1        | 2        |
| 1        | 3        |
| 1        | 4        |
| 2        | 3        |
| 2        | 4        |
| 2        | 5        |
| 6        | 1        |

**Likes table:**
| user_id | page_id |
|---------|---------|
| 1       | 88      |
| 2       | 23      |
| 3       | 24      |
| 4       | 56      |
| 5       | 11      |
| 6       | 33      |
| 2       | 77      |
| 3       | 77      |
| 6       | 88      |

**Output:** 
| recommended_page |
|------------------|
| 23               |
| 24               |
| 56               |
| 33               |
| 77               |

**Explanation:** 
- User one is friend with users 2, 3, 4 and 6.
- Suggested pages are 23 from user 2, 24 from user 3, 56 from user 3 and 33 from user 6.
- Page 77 is suggested from both user 2 and user 3.
- Page 88 is not suggested because user 1 already likes it.

In [0]:
friendship_data_1264 = [
    (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (2, 5), (6, 1)
]
friendship_columns_1264 = ["user1_id", "user2_id"]
friendship_df_1264 = spark.createDataFrame(friendship_data_1264, friendship_columns_1264)
friendship_df_1264.show()

likes_data_1264 = [
    (1, 88), (2, 23), (3, 24), (4, 56), (5, 11), (6, 33),
    (2, 77), (3, 77), (6, 88)
]
likes_columns_1264 = ["user_id", "page_id"]
likes_df_1264 = spark.createDataFrame(likes_data_1264, likes_columns_1264)
likes_df_1264.show()

+--------+--------+
|user1_id|user2_id|
+--------+--------+
|       1|       2|
|       1|       3|
|       1|       4|
|       2|       3|
|       2|       4|
|       2|       5|
|       6|       1|
+--------+--------+

+-------+-------+
|user_id|page_id|
+-------+-------+
|      1|     88|
|      2|     23|
|      3|     24|
|      4|     56|
|      5|     11|
|      6|     33|
|      2|     77|
|      3|     77|
|      6|     88|
+-------+-------+



In [0]:
friends_df_1264 = friendship_df_1264\
                    .filter((col("user1_id") == 1) | (col("user2_id") == 1)) \
                    .withColumn(
                        "friend_id",
                        when(col("user1_id") == 1, col("user2_id")).otherwise(col("user1_id"))
                    ).select("friend_id")

In [0]:
friends_likes_df_1264 = friends_df_1264\
                    .join(likes_df_1264, friends_df_1264.friend_id == likes_df_1264.user_id) \
                    .select("page_id").distinct()

In [0]:
user_likes_df_1264 = likes_df_1264.filter(col("user_id") == 1).select("page_id")

In [0]:
friends_likes_df_1264\
    .join(user_likes_df_1264, on="page_id", how="left_anti") \
    .withColumnRenamed("page_id", "recommended_page").show()

+----------------+
|recommended_page|
+----------------+
|              23|
|              24|
|              56|
|              33|
|              77|
+----------------+

